In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import pickle
import joblib
from scipy.stats import spearmanr
from scipy import stats
import logging
import os

warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from clv_predictor import EnhancedCLVPredictor

# Additional libraries
try:
    import openpyxl
except ImportError:
    print("Warning: openpyxl not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'openpyxl'])
    import openpyxl

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class CLVConfig:
    """Configuration class for CLV model parameters"""
    
    # Data cleaning parameters
    MIN_PURCHASE_AMOUNT = 0
    MIN_QUANTITY = 0
    
    # Feature engineering parameters
    TRAINING_PERIOD_RATIO = 0.8
    RFM_QUANTILES = 5
    LOOKBACK_MONTHS = 12  # Months to look back for features
    PREDICTION_MONTHS = 6  # Months to predict forward
    
    # Model training parameters
    TEST_SIZE = 0.2
    RANDOM_STATE = 42
    CV_FOLDS = 5
    N_JOBS = -1
    
    # Business parameters
    HIGH_VALUE_THRESHOLD_PERCENTILE = 90
    CHURN_RISK_MULTIPLIER = 2.0

class EnhancedCLVPredictor:
    """Production-ready CLV prediction model with advanced features"""
    
    def __init__(self, config=None):
        self.config = config or CLVConfig()
        self.models = {}
        self.best_model = None
        self.scaler = None
        self.feature_names = []
        self.model_performance = {}
        self.is_fitted = False
        
        logging.info("CLV Predictor initialized")
    
    def load_and_clean_data(self, file_path):
        """Load and clean the retail dataset with enhanced error handling"""
        logging.info(f"Loading data from {file_path}")
        
        try:
            # Check current directory
            current_dir = os.getcwd()
            print(f"Current directory: {current_dir}")
            print(f"Files in current directory: {os.listdir('.')}")
            
            # Try different possible filenames
            possible_files = [
                file_path,
                file_path.replace(' ', '_'),
                file_path.replace(' ', ''),
                'online_retail.xlsx',
                'online retail.xlsx',
                'OnlineRetail.xlsx',
                'Online Retail.xlsx'
            ]
            
            df = None
            loaded_file = None
            
            for filename in possible_files:
                if os.path.exists(filename):
                    loaded_file = filename
                    break
            
            if loaded_file is None:
                # List all Excel files in directory
                excel_files = [f for f in os.listdir('.') if f.endswith(('.xlsx', '.xls'))]
                print(f"Available Excel files: {excel_files}")
                
                if excel_files:
                    print(f"Using the first available Excel file: {excel_files[0]}")
                    loaded_file = excel_files[0]
                else:
                    raise FileNotFoundError("No Excel files found in the current directory")
            
            # Try loading the file
            print(f"Attempting to load: {loaded_file}")
            
            if loaded_file.endswith('.xlsx') or loaded_file.endswith('.xls'):
                try:
                    # Try reading Excel file
                    df = pd.read_excel(loaded_file)
                    logging.info(f"Successfully loaded Excel file: {loaded_file}")
                except Exception as e:
                    print(f"Error reading Excel file: {e}")
                    # Try reading as CSV
                    csv_file = loaded_file.replace('.xlsx', '.csv').replace('.xls', '.csv')
                    if os.path.exists(csv_file):
                        df = pd.read_csv(csv_file)
                        logging.info(f"Loaded CSV version: {csv_file}")
                    else:
                        raise
            else:
                df = pd.read_csv(loaded_file)
            
            if df is None:
                raise ValueError("Failed to load any data file")
            
            logging.info(f"Original dataset shape: {df.shape}")
            logging.info(f"Columns: {list(df.columns)}")
            
            # Data cleaning
            df = self._clean_data(df)
            logging.info(f"Cleaned dataset shape: {df.shape}")
            
            return df
            
        except Exception as e:
            logging.error(f"Error loading data: {e}")
            print(f"Error details: {str(e)}")
            raise
    
   
    
    def create_temporal_split(self, df):
        """Create proper temporal split to prevent data leakage"""
        logging.info("Creating temporal split to prevent data leakage")
        
        # Sort by date
        df_sorted = df.sort_values('InvoiceDate')
        
        # Calculate split date based on configuration
        min_date = df_sorted['InvoiceDate'].min()
        max_date = df_sorted['InvoiceDate'].max()
        total_days = (max_date - min_date).days
        
        # Use lookback period for features and prediction period for targets
        feature_end_date = max_date - timedelta(days=self.config.PREDICTION_MONTHS * 30)
        feature_start_date = feature_end_date - timedelta(days=self.config.LOOKBACK_MONTHS * 30)
        
        logging.info(f"Feature period: {feature_start_date} to {feature_end_date}")
        logging.info(f"Prediction period: {feature_end_date} to {max_date}")
        
        # Split data
        feature_data = df_sorted[
            (df_sorted['InvoiceDate'] >= feature_start_date) & 
            (df_sorted['InvoiceDate'] <= feature_end_date)
        ]
        
        target_data = df_sorted[
            df_sorted['InvoiceDate'] > feature_end_date
        ]
        
        logging.info(f"Feature data shape: {feature_data.shape}")
        logging.info(f"Target data shape: {target_data.shape}")
        
        return feature_data, target_data
        
    def _clean_data(self, df):
        """Enhanced data cleaning with better validation"""
    
        # Print column names for debugging
        print(f"Available columns: {list(df.columns)}")
    
        # Try to identify correct column names (case-insensitive and space-insensitive)
        column_mapping = {}
        for col in df.columns:
            col_lower = str(col).lower().strip().replace(' ', '').replace('_', '')
        
            # More flexible column mapping
            if 'customer' in col_lower and 'id' in col_lower:
                column_mapping[col] = 'CustomerID'
            elif 'invoice' in col_lower and 'date' in col_lower:
                column_mapping[col] = 'InvoiceDate'  
            elif col_lower == 'quantity':
                column_mapping[col] = 'Quantity'
            elif ('unit' in col_lower and 'price' in col_lower) or col_lower == 'price':
                column_mapping[col] = 'UnitPrice'
            elif 'invoice' in col_lower and ('no' in col_lower or 'number' in col_lower):
                column_mapping[col] = 'InvoiceNo'
            elif 'stock' in col_lower and 'code' in col_lower:
                column_mapping[col] = 'StockCode'
            elif 'description' in col_lower:
                column_mapping[col] = 'Description'
            elif col_lower == 'country':
                column_mapping[col] = 'Country'
    
        # Apply column mapping
        if column_mapping:
            print(f"Column mapping: {column_mapping}")
            df = df.rename(columns=column_mapping)
            print(f"Columns after mapping: {list(df.columns)}")
    
        # Check required columns exist
        required_columns = ['CustomerID', 'InvoiceDate', 'Quantity', 'UnitPrice']
        missing_columns = [col for col in required_columns if col not in df.columns]
    
        if missing_columns:
            logging.error(f"Missing required columns: {missing_columns}")
            logging.info(f"Available columns: {list(df.columns)}")
        
            # Try to suggest similar columns
            for missing_col in missing_columns:
                similar_cols = [col for col in df.columns if missing_col.lower() in col.lower()]
                if similar_cols:
                    print(f"Similar columns for {missing_col}: {similar_cols}")
        
            raise ValueError(f"Missing required columns: {missing_columns}")
    
        # Remove rows with missing CustomerID
        initial_rows = len(df)
        df = df.dropna(subset=['CustomerID'])
        logging.info(f"Removed {initial_rows - len(df)} rows with missing CustomerID")
        
        # Remove negative quantities and unit prices (returns and adjustments)
        df = df[(df['Quantity'] > self.config.MIN_QUANTITY) & 
                (df['UnitPrice'] > self.config.MIN_PURCHASE_AMOUNT)]
    
        # Convert data types
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
        df = df.dropna(subset=['InvoiceDate'])  # Remove rows with invalid dates
    
        df['CustomerID'] = df['CustomerID'].astype(str)
    
        # Create TotalAmount column
        df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
    
        # Remove outliers (transactions > 99.9th percentile)
        upper_limit = df['TotalAmount'].quantile(0.999)
        df = df[df['TotalAmount'] <= upper_limit]
    
        # Extract enhanced date components
        df['Year'] = df['InvoiceDate'].dt.year
        df['Month'] = df['InvoiceDate'].dt.month
        df['Quarter'] = df['InvoiceDate'].dt.quarter
        df['DayOfWeek'] = df['InvoiceDate'].dt.day_of_week
        df['Hour'] = df['InvoiceDate'].dt.hour
        df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)
    
        # Add seasonality
        df['Season'] = df['Month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                                       3: 'Spring', 4: 'Spring', 5: 'Spring',
                                       6: 'Summer', 7: 'Summer', 8: 'Summer',
                                       9: 'Fall', 10: 'Fall', 11: 'Fall'})
    
        logging.info(f"Data date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")
        logging.info(f"Number of unique customers: {df['CustomerID'].nunique()}")
    
        return df

    def create_advanced_features(self, df, reference_date=None):
        """Create comprehensive features for CLV prediction"""
        logging.info("Creating advanced features")
        
        if reference_date is None:
            reference_date = df['InvoiceDate'].max()
        
        # Basic aggregations
        customer_features = df.groupby('CustomerID').agg({
            'InvoiceDate': ['min', 'max', 'count'],
            'Invoice': 'nunique',
            'TotalAmount': ['sum', 'mean', 'std', 'min', 'max', 'median'],
            'Quantity': ['sum', 'mean', 'std'],
            'StockCode': 'nunique',
            'UnitPrice': ['mean', 'std'],
            'Year': 'nunique',
            'Month': 'nunique',
            'Quarter': 'nunique',
            'DayOfWeek': lambda x: x.mode().iloc[0] if not x.empty else 0,
            'Hour': lambda x: x.mode().iloc[0] if not x.empty else 0,
            'IsWeekend': 'mean',
            'Season': lambda x: x.mode().iloc[0] if not x.empty else 'Spring'
        }).reset_index()
        
        # Flatten column names
        customer_features.columns = [
            'CustomerID', 'FirstPurchase', 'LastPurchase', 'TotalTransactions',
            'UniqueInvoices', 'TotalRevenue', 'AvgOrderValue', 'StdOrderValue',
            'MinOrderValue', 'MaxOrderValue', 'MedianOrderValue', 'TotalQuantity', 
            'AvgQuantity', 'StdQuantity', 'UniqueProducts', 'AvgUnitPrice', 
            'StdUnitPrice', 'YearsActive', 'MonthsActive', 'QuartersActive',
            'PreferredDayOfWeek', 'PreferredHour', 'WeekendPurchaseRate',
            'PreferredSeason'
        ]
        
        # Calculate derived features
        customer_features['CustomerLifespan'] = (
            customer_features['LastPurchase'] - customer_features['FirstPurchase']
        ).dt.days
        
        customer_features['DaysSinceLastPurchase'] = (
            reference_date - customer_features['LastPurchase']
        ).dt.days
        
        customer_features['AvgDaysBetweenPurchases'] = (
            customer_features['CustomerLifespan'] / 
            (customer_features['TotalTransactions'] - 1).clip(lower=1)
        )
        
        # Advanced behavioral features
        customer_features['PurchaseFrequency'] = (
            customer_features['TotalTransactions'] / 
            (customer_features['CustomerLifespan'] + 1).clip(lower=1)
        )
        
        customer_features['PurchaseVelocity'] = (
            customer_features['TotalTransactions'] / 
            customer_features['CustomerLifespan'].clip(lower=1)
        )
        
        customer_features['ProductDiversityRatio'] = (
            customer_features['UniqueProducts'] / customer_features['TotalTransactions']
        )
        
        customer_features['SpendingConsistency'] = (
            1 / (1 + customer_features['StdOrderValue'] / 
                 customer_features['AvgOrderValue'].clip(lower=0.01))
        )
        
        customer_features['RevenuePerTransaction'] = (
            customer_features['TotalRevenue'] / customer_features['TotalTransactions']
        )
        
        customer_features['RevenueGrowthPotential'] = (
            customer_features['MaxOrderValue'] / customer_features['AvgOrderValue']
        )
        
        # RFM Analysis
        customer_features['Recency'] = customer_features['DaysSinceLastPurchase']
        customer_features['Frequency'] = customer_features['TotalTransactions']
        customer_features['Monetary'] = customer_features['TotalRevenue']
        
        # RFM Scores with better handling of edge cases
        try:
            customer_features['RecencyScore'] = pd.qcut(
                customer_features['Recency'], self.config.RFM_QUANTILES, 
                labels=[5,4,3,2,1], duplicates='drop'
            )
        except ValueError:
            # Fallback for when there aren't enough unique values
            customer_features['RecencyScore'] = pd.cut(
                customer_features['Recency'], self.config.RFM_QUANTILES, 
                labels=[5,4,3,2,1], duplicates='drop'
            )
        
        try:
            customer_features['FrequencyScore'] = pd.qcut(
                customer_features['Frequency'].rank(method='first'), 
                self.config.RFM_QUANTILES, labels=[1,2,3,4,5], duplicates='drop'
            )
        except ValueError:
            customer_features['FrequencyScore'] = pd.cut(
                customer_features['Frequency'], self.config.RFM_QUANTILES, 
                labels=[1,2,3,4,5], duplicates='drop'
            )
        
        try:
            customer_features['MonetaryScore'] = pd.qcut(
                customer_features['Monetary'], self.config.RFM_QUANTILES, 
                labels=[1,2,3,4,5], duplicates='drop'
            )
        except ValueError:
            customer_features['MonetaryScore'] = pd.cut(
                customer_features['Monetary'], self.config.RFM_QUANTILES, 
                labels=[1,2,3,4,5], duplicates='drop'
            )
        
        # Convert to numeric
        customer_features['RecencyScore'] = pd.to_numeric(customer_features['RecencyScore'], errors='coerce')
        customer_features['FrequencyScore'] = pd.to_numeric(customer_features['FrequencyScore'], errors='coerce')
        customer_features['MonetaryScore'] = pd.to_numeric(customer_features['MonetaryScore'], errors='coerce')
        
        # Churn risk analysis
        churn_threshold = customer_features['AvgDaysBetweenPurchases'] * self.config.CHURN_RISK_MULTIPLIER
        customer_features['ChurnRisk'] = (
            customer_features['DaysSinceLastPurchase'] > churn_threshold
        ).astype(int)
        
        customer_features['ChurnProbability'] = np.minimum(
            customer_features['DaysSinceLastPurchase'] / churn_threshold.clip(lower=1), 1.0
        )
        
        # Seasonality features
        season_encoder = LabelEncoder()
        customer_features['SeasonEncoded'] = season_encoder.fit_transform(
            customer_features['PreferredSeason'].astype(str)
        )
        
        # Fill missing values
        numeric_columns = customer_features.select_dtypes(include=[np.number]).columns
        customer_features[numeric_columns] = customer_features[numeric_columns].fillna(0)
        
        logging.info(f"Created features for {len(customer_features)} customers")
        return customer_features
    
    def prepare_target_variable(self, feature_data, target_data):
        """Create CLV target variable from future period"""
        logging.info("Preparing target variable")
        
        # Calculate future CLV from target period
        future_clv = target_data.groupby('CustomerID')['TotalAmount'].sum().reset_index()
        future_clv.columns = ['CustomerID', 'FutureCLV']
        
        # Get customers from feature period
        feature_customers = feature_data['CustomerID'].unique()
        
        # Create features from historical data only
        features = self.create_advanced_features(feature_data)
        
        # Merge with future CLV
        clv_data = features.merge(future_clv, on='CustomerID', how='left')
        clv_data['FutureCLV'] = clv_data['FutureCLV'].fillna(0)
        
        logging.info(f"Customers with future purchases: {(clv_data['FutureCLV'] > 0).sum()}")
        logging.info(f"Customers with no future purchases: {(clv_data['FutureCLV'] == 0).sum()}")
        
        return clv_data
    
    def prepare_features_for_modeling(self, df):
        """Prepare features for machine learning"""
        
        # Select features for modeling
        feature_cols = [
            'TotalTransactions', 'UniqueInvoices', 'TotalRevenue', 'AvgOrderValue',
            'StdOrderValue', 'MinOrderValue', 'MaxOrderValue', 'MedianOrderValue',
            'TotalQuantity', 'AvgQuantity', 'StdQuantity', 'UniqueProducts',
            'AvgUnitPrice', 'StdUnitPrice', 'CustomerLifespan', 'DaysSinceLastPurchase',
            'AvgDaysBetweenPurchases', 'PurchaseFrequency', 'PurchaseVelocity',
            'ProductDiversityRatio', 'SpendingConsistency', 'RevenuePerTransaction',
            'RevenueGrowthPotential', 'YearsActive', 'MonthsActive', 'QuartersActive',
            'PreferredDayOfWeek', 'PreferredHour', 'WeekendPurchaseRate',
            'RecencyScore', 'FrequencyScore', 'MonetaryScore', 'ChurnRisk',
            'ChurnProbability', 'SeasonEncoded'
        ]
        
        # Ensure all features exist
        available_features = [col for col in feature_cols if col in df.columns]
        self.feature_names = available_features
        
        X = df[available_features].copy()
        y = df['FutureCLV'].copy()
        
        # Handle missing values
        X = X.fillna(X.median())
        
        # Replace infinite values
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.median())
        
        logging.info(f"Feature matrix shape: {X.shape}")
        logging.info(f"Target variable shape: {y.shape}")
        
        return X, y
    
    def calculate_business_metrics(self, y_true, y_pred, customer_ids=None):
        """Calculate business-focused evaluation metrics"""
        
        # Standard regression metrics
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        # Business-focused metrics
        ranking_corr = spearmanr(y_true, y_pred)[0] if len(y_true) > 1 else 0
        
        # Top customer identification precision
        if len(y_true) > 10:
            top_10_pct = max(1, int(len(y_true) * 0.1))
            true_top_indices = y_true.argsort()[-top_10_pct:]
            pred_top_indices = y_pred.argsort()[-top_10_pct:]
            top_10_precision = len(set(true_top_indices) & set(pred_top_indices)) / top_10_pct
        else:
            top_10_precision = 0
        
        # MAPE (Mean Absolute Percentage Error) for non-zero values
        non_zero_mask = y_true > 0
        if non_zero_mask.sum() > 0:
            mape = mean_absolute_percentage_error(y_true[non_zero_mask], y_pred[non_zero_mask])
        else:
            mape = np.inf
        
        return {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'MAPE': mape,
            'Ranking_Correlation': ranking_corr,
            'Top_10_Precision': top_10_precision
        }
    
    def create_model_pipeline(self):
        """Create advanced model pipeline with preprocessing"""
        
        # Define models with hyperparameter grids (reduced for faster training)
        models = {
            'Linear_Regression': {
                'model': LinearRegression(),
                'params': {},
                'scale': True
            },
            'Ridge_Regression': {
                'model': Ridge(),
                'params': {'alpha': [0.1, 1.0, 10.0]},
                'scale': True
            },
            'Random_Forest': {
                'model': RandomForestRegressor(random_state=self.config.RANDOM_STATE, n_jobs=2),
                'params': {
                    'n_estimators': [100, 200],
                    'max_depth': [10, 20, None],
                    'min_samples_split': [2, 5],
                    'min_samples_leaf': [1, 2]
                },
                'scale': False
            },
            'Gradient_Boosting': {
                'model': GradientBoostingRegressor(random_state=self.config.RANDOM_STATE),
                'params': {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1],
                    'max_depth': [3, 5]
                },
                'scale': False
            },
            'Extra_Trees': {
                'model': ExtraTreesRegressor(random_state=self.config.RANDOM_STATE, n_jobs=2),
                'params': {
                    'n_estimators': [100, 200],
                    'max_depth': [10, 20],
                    'min_samples_split': [2, 5]
                },
                'scale': False
            }
        }
        
        return models
    
    def train_and_evaluate_models(self, X_train, X_test, y_train, y_test):
        """Train multiple models with hyperparameter tuning"""
        logging.info("Training and evaluating models with hyperparameter tuning")
        
        models = self.create_model_pipeline()
        results = []
        
        # Initialize scalers
        standard_scaler = StandardScaler()
        
        for name, model_config in models.items():
            logging.info(f"Training {name}...")
            
            try:
                model = model_config['model']
                params = model_config['params']
                needs_scaling = model_config['scale']
                
                # Prepare data
                if needs_scaling:
                    X_train_processed = standard_scaler.fit_transform(X_train)
                    X_test_processed = standard_scaler.transform(X_test)
                else:
                    X_train_processed = X_train
                    X_test_processed = X_test
                
                # Hyperparameter tuning
                if params:
                    grid_search = GridSearchCV(
                        model, params, cv=3,  # Reduced CV folds for speed
                        scoring='r2', n_jobs=2, verbose=0
                    )
                    grid_search.fit(X_train_processed, y_train)
                    best_model = grid_search.best_estimator_
                    best_params = grid_search.best_params_
                else:
                    best_model = model
                    best_model.fit(X_train_processed, y_train)
                    best_params = {}
                
                # Make predictions
                y_pred_train = best_model.predict(X_train_processed)
                y_pred_test = best_model.predict(X_test_processed)
                
                # Calculate metrics
                train_metrics = self.calculate_business_metrics(y_train, y_pred_train)
                test_metrics = self.calculate_business_metrics(y_test, y_pred_test)
                
                # Cross-validation
                cv_scores = cross_val_score(
                    best_model, X_train_processed, y_train, 
                    cv=3, scoring='r2'
                )
                
                # Store results
                result = {
                    'Model': name,
                    'Best_Params': str(best_params),
                    'Train_R2': train_metrics['R2'],
                    'Test_R2': test_metrics['R2'],
                    'Train_RMSE': train_metrics['RMSE'],
                    'Test_RMSE': test_metrics['RMSE'],
                    'Test_MAE': test_metrics['MAE'],
                    'Test_MAPE': test_metrics['MAPE'] if test_metrics['MAPE'] != np.inf else 999,
                    'Ranking_Correlation': test_metrics['Ranking_Correlation'],
                    'Top_10_Precision': test_metrics['Top_10_Precision'],
                    'CV_R2_Mean': cv_scores.mean(),
                    'CV_R2_Std': cv_scores.std(),
                    'Needs_Scaling': needs_scaling
                }
                
                results.append(result)
                
                # Store trained model
                self.models[name] = {
                    'model': best_model,
                    'scaler': standard_scaler if needs_scaling else None,
                    'params': best_params
                }
                
                logging.info(f"Completed {name} - R2: {test_metrics['R2']:.4f}")
                
            except Exception as e:
                logging.error(f"Error training {name}: {e}")
                continue
        
        # Convert to DataFrame and find best model
        results_df = pd.DataFrame(results)
        if not results_df.empty:
            best_idx = results_df['Test_R2'].idxmax()
            best_model_name = results_df.loc[best_idx, 'Model']
            self.best_model = self.models[best_model_name]
            self.is_fitted = True
            
            logging.info(f"Best model: {best_model_name} with R2: {results_df.loc[best_idx, 'Test_R2']:.4f}")
        
        return results_df
    
    def analyze_feature_importance(self, model_name=None):
        """Analyze feature importance for tree-based models"""
        
        if model_name is None and self.models:
            model_name = list(self.models.keys())[0]
        
        model_info = self.models.get(model_name)
        if not model_info:
            return None
        
        model = model_info['model']
        
        # Feature importance for tree-based models
        if hasattr(model, 'feature_importances_'):
            importance_df = pd.DataFrame({
                'Feature': self.feature_names,
                'Importance': model.feature_importances_
            }).sort_values('Importance', ascending=False)
            
            return importance_df
        
        return None
    
    def predict_clv(self, X):
        """Make CLV predictions using the best model"""
        
        if not self.is_fitted:
            raise ValueError("Model must be fitted before making predictions")
        
        model_info = self.best_model
        model = model_info['model']
        scaler = model_info['scaler']
        
        # Prepare data
        if scaler is not None:
            X_processed = scaler.transform(X)
        else:
            X_processed = X
        
        predictions = model.predict(X_processed)
        return predictions
    
    def segment_customers(self, predictions, percentiles=[20, 40, 60, 80]):
        """Segment customers based on predicted CLV"""
        
        segments = pd.cut(predictions, 
                         bins=np.percentile(predictions, [0] + percentiles + [100]),
                         labels=['Low Value', 'Low-Medium', 'Medium', 'Medium-High', 'High Value'],
                         include_lowest=True)
        
        return segments
    
    def generate_business_insights(self, X, y_true, y_pred, segments):
        """Generate comprehensive business insights"""
        
        insights = {}
        
        # Overall model performance
        metrics = self.calculate_business_metrics(y_true, y_pred)
        insights['model_performance'] = metrics
        
        # Segment analysis
        segment_stats = pd.DataFrame({
            'Segment': segments,
            'Predicted_CLV': y_pred,
            'Actual_CLV': y_true
        }).groupby('Segment').agg({
            'Predicted_CLV': ['count', 'mean', 'median', 'std'],
            'Actual_CLV': ['mean', 'median', 'std']
        }).round(2)
        
        insights['segment_analysis'] = segment_stats
        
        # Revenue impact
        total_predicted_clv = y_pred.sum()
        high_value_mask = segments == 'High Value'
        high_value_clv = y_pred[high_value_mask].sum() if high_value_mask.sum() > 0 else 0
        
        insights['revenue_impact'] = {
            'total_predicted_clv': total_predicted_clv,
            'high_value_contribution': high_value_clv,
            'high_value_percentage': (high_value_clv / total_predicted_clv * 100) if total_predicted_clv > 0 else 0
        }
        
        return insights
    
    def save_model(self, filepath):
        """Save the trained model and preprocessing components"""
        
        if not self.is_fitted:
            logging.warning("Model hasn't been fitted yet. Saving current state anyway.")
        
        model_package = {
            'models': self.models,
            'best_model': self.best_model,
            'feature_names': self.feature_names,
            'model_performance': self.model_performance,
            'config': self.config,
            'is_fitted': self.is_fitted,
            'scaler': self.scaler
        }
        
        try:
            # Save using joblib for better sklearn model serialization
            joblib.dump(model_package, filepath)
            logging.info(f"Model saved successfully to {filepath}")
        except Exception as e:
            logging.error(f"Error saving model: {e}")
            raise
    
    def load_model(self, filepath):
        """Load a pre-trained model and preprocessing components"""
        
        try:
            model_package = joblib.load(filepath)
            
            self.models = model_package['models']
            self.best_model = model_package['best_model']
            self.feature_names = model_package['feature_names']
            self.model_performance = model_package['model_performance']
            self.config = model_package['config']
            self.is_fitted = model_package['is_fitted']
            self.scaler = model_package.get('scaler', None)
            
            logging.info(f"Model loaded successfully from {filepath}")
            
        except Exception as e:
            logging.error(f"Error loading model: {e}")
            raise
    
    def run_full_pipeline(self, file_path, save_model_path=None):
        """Run the complete CLV prediction pipeline"""
        
        try:
            # Load and clean data
            df = self.load_and_clean_data(file_path)
            
            # Create temporal split
            feature_data, target_data = self.create_temporal_split(df)
            
            # Prepare features and target
            clv_data = self.prepare_target_variable(feature_data, target_data)
            X, y = self.prepare_features_for_modeling(clv_data)
            
            # Split data for training
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.config.TEST_SIZE, 
                random_state=self.config.RANDOM_STATE
            )
            
            # Train models
            results = self.train_and_evaluate_models(X_train, X_test, y_train, y_test)
            
            # Make predictions
            y_pred = self.predict_clv(X_test)
            
            # Customer segmentation
            segments = self.segment_customers(y_pred)
            
            # Generate insights
            insights = self.generate_business_insights(X_test, y_test, y_pred, segments)
            
            # Save model if path provided
            if save_model_path:
                self.save_model(save_model_path)
            
            return {
                'model_results': results,
                'predictions': y_pred,
                'segments': segments,
                'insights': insights,
                'feature_importance': self.analyze_feature_importance()
            }
            
        except Exception as e:
            logging.error(f"Error in pipeline: {e}")
            raise

# Example usage function
def main():
    """Main function to demonstrate CLV prediction"""
    
    # Initialize predictor
    predictor = EnhancedCLVPredictor()
    
    # File path - try different variations
    possible_files = [
        'online retail.xlsx',
        'online_retail.xlsx',
        'OnlineRetail.xlsx',
        'Online Retail.xlsx'
    ]
    
    file_path = None
    for file in possible_files:
        if os.path.exists(file):
            file_path = file
            break
    
    if file_path is None:
        print("Error: Excel file not found. Please ensure your file is named correctly.")
        print("Expected names: 'online retail.xlsx', 'online_retail.xlsx', etc.")
        return
    
    try:
        # Run pipeline
        results = predictor.run_full_pipeline(file_path, save_model_path='clv_model.pkl')
        
        # Display results
        print("\n=== MODEL RESULTS ===")
        print(results['model_results'])
        
        print("\n=== FEATURE IMPORTANCE ===")
        if results['feature_importance'] is not None:
            print(results['feature_importance'].head(10))
        
        print("\n=== BUSINESS INSIGHTS ===")
        print(f"Model Performance: {results['insights']['model_performance']}")
        print(f"Revenue Impact: {results['insights']['revenue_impact']}")
        
        # Plot results
        plt.figure(figsize=(15, 10))
        
        # Model comparison
        plt.subplot(2, 2, 1)
        results['model_results'].plot(x='Model', y='Test_R2', kind='bar', ax=plt.gca())
        plt.title('Model Performance Comparison')
        plt.xticks(rotation=45)
        
        # Feature importance
        if results['feature_importance'] is not None:
            plt.subplot(2, 2, 2)
            top_features = results['feature_importance'].head(10)
            plt.barh(top_features['Feature'], top_features['Importance'])
            plt.title('Top 10 Feature Importance')
        
        # Customer segments
        plt.subplot(2, 2, 3)
        segment_counts = pd.Series(results['segments']).value_counts()
        segment_counts.plot(kind='pie', ax=plt.gca())
        plt.title('Customer Segments Distribution')
        
        # Prediction vs Actual
        plt.subplot(2, 2, 4)
        plt.scatter(results['insights']['model_performance'], results['predictions'][:100])
        plt.xlabel('Actual CLV')
        plt.ylabel('Predicted CLV')
        plt.title('Predictions vs Actual (Sample)')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error running pipeline: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

2025-06-26 16:58:44,837 - INFO - CLV Predictor initialized
2025-06-26 16:58:44,841 - INFO - Loading data from OnlineRetail.xlsx


Current directory: C:\Users\krati\celebal Project
Files in current directory: ['.ipynb_checkpoints', 'app.py', 'new.ipynb', 'OnlineRetail.xlsx', 'simple_test.py', 'Untitled.ipynb']
Attempting to load: OnlineRetail.xlsx


2025-06-26 16:59:22,788 - INFO - Successfully loaded Excel file: OnlineRetail.xlsx
2025-06-26 16:59:22,790 - INFO - Original dataset shape: (525461, 8)
2025-06-26 16:59:22,790 - INFO - Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


Available columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']
Column mapping: {'StockCode': 'StockCode', 'Description': 'Description', 'Quantity': 'Quantity', 'InvoiceDate': 'InvoiceDate', 'Price': 'UnitPrice', 'Customer ID': 'CustomerID', 'Country': 'Country'}
Columns after mapping: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


2025-06-26 16:59:22,878 - INFO - Removed 107927 rows with missing CustomerID
2025-06-26 16:59:23,435 - INFO - Data date range: 2009-12-01 07:45:00 to 2010-12-09 20:01:00
2025-06-26 16:59:23,467 - INFO - Number of unique customers: 4304
2025-06-26 16:59:23,478 - INFO - Cleaned dataset shape: (407259, 16)
2025-06-26 16:59:23,480 - INFO - Creating temporal split to prevent data leakage
2025-06-26 16:59:23,564 - INFO - Feature period: 2009-06-17 20:01:00 to 2010-06-12 20:01:00
2025-06-26 16:59:23,565 - INFO - Prediction period: 2010-06-12 20:01:00 to 2010-12-09 20:01:00
2025-06-26 16:59:23,852 - INFO - Feature data shape: (176904, 16)
2025-06-26 16:59:23,854 - INFO - Target data shape: (230355, 16)
2025-06-26 16:59:23,870 - INFO - Preparing target variable
2025-06-26 16:59:23,934 - INFO - Creating advanced features
2025-06-26 16:59:24,995 - INFO - Created features for 2815 customers
2025-06-26 16:59:25,010 - INFO - Customers with future purchases: 1992
2025-06-26 16:59:25,012 - INFO - Cust